# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
import numpy as np

import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sqlalchemy import create_engine
import sqlite3


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bernardogarcia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bernardogarcia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bernardogarcia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Note** - We want to classify the messages into categories. `messages` will be the input samples (X) and `categories` (y) will be the predicted classes.

In [2]:
# Function to load data from database
def load_data():
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql_table(table_name='InsertTableName', con=engine)
    X = df['message'].values
    y = df.drop(['id', 'message', 'original', 'genre'], axis=1).values
    return X, y, df

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    ''' This function takes an array with the messages and returns an array with the tokenized text.
    
    INPUT:
    
    text - Messages in the df stored in the database (array-like object, string)
    
    OUTPUT:
    
    lemmed_text - Tokenized and lemmed messages in the df stored in the database (array-like object, string)
    '''
    
    # Normalize text to all lower case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]"," ", text)
    lower_case_text = text.lower()
    
    # Tokenize text
    tokens = word_tokenize(lower_case_text)
    
    # Remove stopwords
    words = [w for w in tokens if w not in stopwords.words("english")]
    
    # Instantiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Reduce words to their root form
    lemmed_text = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
        
    return lemmed_text
    

In [4]:
X, y, df = load_data()

# test out function
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['look', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroy', 'hospital', 'st', 'croix', 'function', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train pipeline on train set
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
y_test.shape

(7854, 36)

In [19]:
y_pred.shape

(7854, 36)

In [20]:
# Category labels

category_labels = list(df.columns[4:])

In [21]:
from sklearn.metrics import classification_report

for i, label in zip(range(y_pred.shape[1]), category_labels):
        print('Category:', label, classification_report(y_test[:, i], y_pred[:, i],
                                                        target_names=['Negative', 'Positive'],labels=[0, 1]))

              precision    recall  f1-score   support

    Negative       0.52      0.47      0.49      1869
    Positive       0.83      0.87      0.85      5926

   micro avg       0.77      0.77      0.77      7795
   macro avg       0.68      0.67      0.67      7795
weighted avg       0.76      0.77      0.76      7795

              precision    recall  f1-score   support

    Negative       0.87      0.98      0.92      6512
    Positive       0.74      0.28      0.41      1342

    accuracy                           0.86      7854
   macro avg       0.80      0.63      0.66      7854
weighted avg       0.85      0.86      0.83      7854

              precision    recall  f1-score   support

    Negative       1.00      1.00      1.00      7816
    Positive       0.00      0.00      0.00        38

    accuracy                           1.00      7854
   macro avg       0.50      0.50      0.50      7854
weighted avg       0.99      1.00      0.99      7854

              preci

/Users/bernardogarcia/opt/anaconda3/envs/drp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bernardogarcia/opt/anaconda3/envs/drp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    Negative       0.99      1.00      1.00      7813
    Positive       0.00      0.00      0.00        41

    accuracy                           0.99      7854
   macro avg       0.50      0.50      0.50      7854
weighted avg       0.99      0.99      0.99      7854

              precision    recall  f1-score   support

    Negative       0.99      1.00      0.99      7763
    Positive       0.00      0.00      0.00        91

    accuracy                           0.99      7854
   macro avg       0.49      0.50      0.50      7854
weighted avg       0.98      0.99      0.98      7854

              precision    recall  f1-score   support

    Negative       0.99      1.00      1.00      7802
    Positive       0.00      0.00      0.00        52

    accuracy                           0.99      7854
   macro avg       0.50      0.50      0.50      7854
weighted avg       0.99      0.99      0.99      7854

              preci

In [30]:
# Simple example to understand classification_report output
y_real = [1, 0, 0, 1, 1]
y_predic = [1, 0, 1, 0, 1]

In [31]:
print('Class 1', classification_report(y_predic, y_real, target_names=['Negative', 'Positive'],labels=[0, 1]))

Class 1               precision    recall  f1-score   support

    Negative       0.50      0.50      0.50         2
    Positive       0.67      0.67      0.67         3

    accuracy                           0.60         5
   macro avg       0.58      0.58      0.58         5
weighted avg       0.60      0.60      0.60         5



### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
# Display parameters in the pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x106857ca0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=KNeighborsClassifier(), n_jobs=-1))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x106857ca0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=KNeighborsClassifier(), n_jobs=-1),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': Fal

In [14]:
#'tfidf__norm': ()'l2'),
    #'tfidf__smooth_idf': (True, False),
    #'tfidf__sublinear_tf': (True, False),

In [25]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'clf__estimator__n_neighbors': [2, 5],
    'clf__estimator__weights': ('uniform', 'distance')  
}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x106857ca0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier(),
                                                              n_jobs=-1))]),
             param_grid={'clf__estimator__n_neighbors': [2, 5],
                         'clf__estimator__weights': ('uniform', 'distance')})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [45]:
# Predict on test data
y_pred = cv.predict(X_test)


In [43]:
# Print classification_report
category_labels = list(df.columns[4:])

for i, label in zip(range(y_pred.shape[1]), category_labels):
        print('Category:', label,'\n', classification_report(y_test[:, i], y_pred[:, i],
                                                        target_names=['Negative', 'Positive'],labels=[0, 1]))

Category: related 
               precision    recall  f1-score   support

    Negative       0.52      0.47      0.49      1869
    Positive       0.83      0.87      0.85      5926

   micro avg       0.77      0.77      0.77      7795
   macro avg       0.68      0.67      0.67      7795
weighted avg       0.76      0.77      0.76      7795

Category: request 
               precision    recall  f1-score   support

    Negative       0.87      0.98      0.92      6512
    Positive       0.74      0.28      0.41      1342

    accuracy                           0.86      7854
   macro avg       0.80      0.63      0.66      7854
weighted avg       0.85      0.86      0.83      7854

Category: offer 
               precision    recall  f1-score   support

    Negative       1.00      1.00      1.00      7816
    Positive       0.00      0.00      0.00        38

    accuracy                           1.00      7854
   macro avg       0.50      0.50      0.50      7854
weighted avg    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# TODO

### 9. Export your model as a pickle file

In [ ]:
# Save the model to disk
pkl_filename = 'KNeighborsClassifier.pkl'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.